In [6]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.io as pio
import re

In [7]:
# задать количество столбцов и строк отображения датафрейма
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 50)

In [8]:
# # 1. Стиль фона и сетки
# sns.set_style("whitegrid")  # или "darkgrid", "ticks"

# # 2. Палитра цветов (единая для всех категориальных графиков)
# sns.set_palette("magma")    # или "viridis", "rocket", "Blues", "Set2" и т.д.

# # 3. Размер шрифтов по умолчанию
# plt.rcParams.update({
#     'font.size': 12,
#     'axes.titlesize': 16,      # заголовок графика
#     'axes.labelsize': 13,      # подписи осей X и Y
#     'xtick.labelsize': 11,     # метки на оси X
#     'ytick.labelsize': 11,     # метки на оси Y
#     'legend.fontsize': 11,
#     'figure.figsize': (10, 6), # размер фигуры по умолчанию
#     'figure.dpi': 100,         # качество изображения
#     'axes.spines.top': False,  # убрать верхнюю рамку
#     'axes.spines.right': False # убрать правую рамку
# })

In [9]:
# тема для графиков по умалчанию - plotly_white рекомендуется для презентаций
pio.templates.default = "plotly_white"

# Разведочный анализ данных цен аренды квартир в г. Москве

**Дано:** файл с данными в формате csv, в котором содержатся данные по аренде квартир, содержащие информацию о характеристиках квартиры и иные показатели.

**Цель:** провести разведочный анализ данных, содержащихся в файле. Целевой город Москва. Целевая переменная - цена аренды.


In [10]:
# загружаем данные. 2 варианта с google диска или из локальной папки data и выодим на экран
# file_id = "130KYOX8O4wrP_T8vdz2GfvJRQ03ONmE7"
# path_data = f"https://drive.google.com/uc?export=download&id={file_id}"
path_data = r'../../data/_data.csv' # если не работает скачивание с google диска
data_df = pd.read_csv(path_data, encoding='utf-8', index_col=0)
data_df.head(10)

,ID объявления,Количество комнат,Тип,Метро,Адрес,"Площадь, м2",Дом,Парковка,Цена,Телефоны,Описание,Ремонт,"Площадь комнат, м2",Балкон,Окна,Санузел,Можно с детьми/животными,Дополнительно,Название ЖК,Серия дома,"Высота потолков, м",Лифт,Мусоропровод,Ссылка на объявление
0,271271157,4,Квартира,м. Смоленская (9 мин пешком),"Москва, улица Новый Арбат, 27",200.0/20.0,"5/16, Монолитный",подземная,"500000.0 руб./ За месяц, Залог - 500000 руб., ...",+79166369231,Без комиссии для нанимателя! Бонус коллегам 12...,Дизайнерский,NaN,NaN,NaN,NaN,"Можно с детьми, Можно с животными","Мебель в комнатах, Мебель на кухне, Ванна, Душ...","Новый Арбат, 2010",NaN,3.0,"Пасс (4), Груз (1)",Да,https://www.cian.ru/rent/flat/271271157
1,271634126,4,Квартира,м. Смоленская (8 мин пешком),"Москва, улица Новый Арбат, 27",198.0/95.0/18.0,"5/16, Монолитно-кирпичный",подземная,"500000.0 руб./ За месяц, Залог - 500000 руб., ...",+79850608590,Лот 93107. Елена Анисимова.\n\nБонус агенту 50...,Дизайнерский,25 25 20 25,NaN,На улицу и двор,"Совмещенный (2), Раздельный (1)",Можно с детьми,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...",Новый Арбат,NaN,3.5,"Пасс (1), Груз (1)",Нет,https://www.cian.ru/rent/flat/271634126
2,271173086,"4, Оба варианта",Квартира,м. Смоленская (7 мин пешком),"Москва, улица Новый Арбат, 27",200.0/116.0/4.0,5/16,подземная,"500000.0 руб./ За месяц, Залог - 500000 руб., ...","+79672086536, +79099269384","ID 36380: Шикарная 4-х км. квартира в ЖК ""Нов...",Евроремонт,NaN,NaN,На улицу и двор,Совмещенный (3),Можно с детьми,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...",Новый Арбат,NaN,3.2,Пасс (1),NaN,https://www.cian.ru/rent/flat/271173086
3,272197456,"4, Оба варианта",Квартира,м. Смоленская (3 мин пешком),"Москва, переулок Плотников, 21С1",170.0/95.0/17.0,5/6,подземная,"400000.0 руб./ За месяц, Залог - 400000 руб., ...","+79660342340, +79099269384",ID 31618: Эксклюзивное предложение. Современн...,Евроремонт,14-42-20-19,NaN,На улицу и двор,Совмещенный (3),Можно с животными,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...",NaN,NaN,3.2,Пасс (1),NaN,https://www.cian.ru/rent/flat/272197456
4,273614615,2,Квартира,м. Арбатская (7 мин пешком),"Москва, улица Новый Арбат, 15",58.0/38.0/5.0,"12/26, Панельный",NaN,"225000.0 руб./ За месяц, Залог - 225000 руб., ...",+79852432860,Лот 111542. Татьяна Лучкина.\n\nБонус агенту 5...,Евроремонт,20 18,NaN,На улицу и двор,Совмещенный (2),NaN,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...",The Book,NaN,3.9,"Пасс (1), Груз (1)",Да,https://www.cian.ru/rent/flat/273614615
5,274837728,3,Квартира,м. Смоленская (5 мин пешком),"Москва, 1-й Смоленский переулок, 21",92.0,3/7,NaN,"470000.0 руб./ За месяц, Залог - 470000 руб., ...",+79684728732,ID 743. С коллегами работаем 50/50. Видовые ап...,Дизайнерский,NaN,NaN,NaN,NaN,NaN,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...",Smolensky De Luxe,NaN,NaN,NaN,NaN,https://www.cian.ru/rent/flat/274837728
6,273643908,"5, Оба варианта",Квартира,м. Арбатская (4 мин пешком),"Москва, переулок Романов, 3С1",213.0/140.0/20.0,4/5,наземная,"350000.0 руб./ За месяц, Залог - 350000 руб., ...","+79175135482, +79099269384",ID 41980: Предлагается 5-и комнатная квартира...,Евроремонт,43-25-25-25-22,NaN,На улицу и двор,Совмещенный (2),"Можно с детьми, Можно с животными","Мебель в комнатах, Мебель на кухне, Ванна, Сти...",NaN,NaN,3.2,Пасс (1),NaN,https://www.cian.ru/rent/flat/273643908
7,274475342,3,Квартира,м. Смоленская (3 мин пешком),"Москва, улица Арбат, 43С3",98.0/63.0/9.0,"2/4, Монолитный",подземная,"250000.0 руб./ За месяц, Залог - 250000 руб., ...",+79152004882,Лот 112453. Ирина Панченко.\n\nБонус агенту 40...,Евроремонт,26 22 15,NaN,Во двор,"Совмещенный (1), Раздельный (1)","Можно с детьми, Можно с животными","Мебель в комнатах, Мебель на кухне, Ванна, Душ...",NaN,NaN,3.2,Пасс (1),Нет,https://www.cian.ru/rent/flat/274475342
8,273973191,3,Квартира,м. Смоленская (9 мин пешком),"Москва, Новинский бульвар, 18С1",120.0/95.0/10.0,"5/10, Сталинский",открытая,"130000.0 руб./ За месяц,

## Объявления не из Москвы

In [11]:
# проверка, есть ли объявления не из Москвы. Проверка визуально, но можно через shape
print(f"Количество объявлений по аренде квартир не из Москвы - {data_df[~(data_df['Адрес'].str.upper().str.contains('МОСКВА', na=False))].shape[0]}")

Количество объявлений по аренде квартир не из Москвы - 3631


### В данных присутствуют объявления аренды квартир не из Москвы. Оставим только московские объявления

In [12]:
# оставляем только Москву
data_df = data_df[data_df['Адрес'].str.upper().str.contains('МОСКВА', na=False)].copy()

In [13]:
# названия признаков(столбцов), типы данных, количество записей, пропуски
data_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19737 entries, 0 to 23367
Data columns (total 24 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   ID  объявления            19737 non-null  int64  
 1   Количество комнат         19202 non-null  object 
 2   Тип                       19737 non-null  object 
 3   Метро                     19391 non-null  object 
 4   Адрес                     19737 non-null  object 
 5   Площадь, м2               19737 non-null  object 
 6   Дом                       19737 non-null  object 
 7   Парковка                  8563 non-null   object 
 8   Цена                      19737 non-null  object 
 9   Телефоны                  19737 non-null  object 
 10  Описание                  19737 non-null  object 
 11  Ремонт                    17274 non-null  object 
 12  Площадь комнат, м2        12509 non-null  object 
 13  Балкон                    13107 non-null  object 
 14  Окна       

In [14]:
# статистика по всем столбцам
data_df.describe(include='all')

,ID объявления,Количество комнат,Тип,Метро,Адрес,"Площадь, м2",Дом,Парковка,Цена,Телефоны,Описание,Ремонт,"Площадь комнат, м2",Балкон,Окна,Санузел,Можно с детьми/животными,Дополнительно,Название ЖК,Серия дома,"Высота потолков, м",Лифт,Мусоропровод,Ссылка на объявление
count,1.973700e+04,19202,19737,19391,19737,19737,19737,8563,19737,19737,19737,17274,12509,13107,14587,17696,14822,19465,4456,2091,10535.000000,15545,11730,19737
unique,NaN,24,1,4871,10452,9585,2415,5,2219,11616,19456,4,3462,18,3,20,3,488,1133,369,NaN,34,2,19737
top,NaN,1,Квартира,м. Водный стадион (5 мин пешком),"Москва, Чапаевский переулок, 3",40.0/20.0/10.0,"3/5, Кирпичный",наземная,"40000.0 руб./ За месяц, Залог - 40000 руб., Ко...",+79652883064,0,Косметический,20,Балкон (1),Во двор,Совмещенный (1),Можно с детьми,"Мебель в комнатах, Мебель на кухне, Ванна, Сти...","Символ, 2019",индивидуальный проект,NaN,Пасс (1),Да,https://www.cian.ru/rent/flat/271271157
freq,NaN,6646,19737,40,63,214,265,5283,1112,253,108,7361,1500,6336,9708,8500,8540,2553,61,615,NaN,4865,10262,1
mean,2.671149e+08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.992925,NaN,NaN,NaN
std,1.980106e+07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.852740,NaN,NaN,NaN
min,1.072986e+08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.200000,NaN,NaN,NaN
25%,2.712212e+08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.640000,NaN,NaN,NaN
50%,2.739284e+08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.640000,NaN,NaN,NaN
75%,2.746973e+08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.800000,NaN,NaN,NaN


Есть выбросы по `Высота потолков, м` в обе стороны(1.2м и 320м ). Возможно введены неверные/ошибочные данные.

Колонка Тип имеет одно уникальное значение и непредставляет ценности.

в части колонок грязные данные.

In [15]:
# размеры датафрейма
data_df.shape

(19737, 24)

In [16]:
# сколько столбцов по типам
columns_number = data_df.select_dtypes(include=['number']).columns.tolist()
columns_obj = data_df.select_dtypes(include=['object', 'category']).columns.tolist()
columns_bool = data_df.select_dtypes(include=['bool']).columns.tolist()
print(columns_number)
print(f"Колонки по типам:\nnumeric - {len(columns_number)}\nobject - {len(columns_obj)}\nbool - {len(columns_bool)}")

['ID  объявления', 'Высота потолков, м']
Колонки по типам:
numeric - 2
object - 22
bool - 0


## Значения в некоторых столбцах

In [17]:
print(f"Варианты заполнения значений в колонке Количество комнат:\n{data_df['Количество комнат'].unique()}")

Варианты заполнения значений в колонке Количество комнат:
['4' '4, Оба варианта' '2' '3' '5, Оба варианта' '3, Изолированная' '5'
 '5, Изолированная' '2, Оба варианта' '3, Оба варианта' '6' '1'
 '4, Изолированная' '6, Оба варианта' '2, Изолированная' nan '2, Смежная'
 '4, Смежная' '3, Смежная' '6, Изолированная' '1, Изолированная'
 '1, Оба варианта' '5, Смежная' '6, Смежная' '1, Смежная']


In [18]:
print(f"Варианты заполнения значений в колонке Парковка:\n{data_df['Парковка'].unique()}")

Варианты заполнения значений в колонке Парковка:
['подземная' nan 'наземная' 'открытая' 'многоуровневая' 'на крыше']


In [19]:
print(f"Варианты заполнения значений в колонке Парковка:\n{data_df['Ремонт'].unique()}")

Варианты заполнения значений в колонке Парковка:
['Дизайнерский' 'Евроремонт' 'Косметический' nan 'Без ремонта']


In [20]:
print(f"Количество дубликатов по ID  объявления:\n{data_df[data_df['ID  объявления'].duplicated()]['ID  объявления'].sum()}")

Количество дубликатов по ID  объявления:
0


Удаление дубликатов по колонке 'ID  объявления', если они есть 

In [21]:
data_df = data_df.drop_duplicates(subset=['ID  объявления'], keep='first')

In [22]:
# выбираем пропущенные значения из датафрейма
missing_df = pd.DataFrame((data_df.isna().mean() * 100) \
            .sort_values(ascending=False) \
            .round(2) \
            .reset_index()) \
            .rename(columns={
                    'index': 'Название столбца',
                    0: 'Процент пропусков, %'
})

## Анализ пропусков

На графике показан процент пропущенных значений по каждой колонке.  

In [23]:
fig = px.bar(
    missing_df,
    x='Процент пропусков, %',
    y='Название столбца',
    title='Процент пропущенных значений по колонкам',
    orientation='h',
)

# Увеличиваем высоту — чтобы вместить все строки
fig.update_layout(height=800)

fig.show()

In [24]:
# plt.figure(figsize=(10, 8))
# ax = sns.barplot(
#     data=missing_df,
#     x='Процент пропусков, %',
#     y='Название столбца'
# )

# for container in ax.containers:
#     ax.bar_label(container, fmt='%.2f%%', padding=5, fontsize=10)

# plt.title('Процент пропущенных значений по колонкам', fontsize=16, pad=20)
# plt.xlabel('Процент пропусков (%)', fontsize=12)
# plt.ylabel('Колонка', fontsize=12)
# plt.grid(axis='x', linestyle='--', alpha=0.7)
# plt.tight_layout()
# plt.show()

5 признаков имеют количество пропусков более 40%: `Серия дома`, `Название ЖК`, `Парковка`, `Высота потолков`, `Мусоропровод`. 

`Серия дома`, `Название ЖК` имеют пропуски > 70% значения малопригодны для дальнейшей обработки. Рекомендуется удалить.

**Целевая переменная(цена аренды)** находится в ячейке `Цена` среди другой информации. Есть возможность выделить ее.

Также есть возможно выделить числовые признаки для EDA:
- количество комнат из ячейки `Количество комнат`, 
- площадь квартиры из ячейки `Площадь, м2`

## Выделим целевую переменную (цена аренды) и важные пармаетры для цены - площадь квартиры и количество комнат

In [25]:
#  выделение цены - целевой переменной
def get_price_rent(text):
    if pd.isna(text) or str(text).strip() == '':
        return np.nan
    
    price_get = str(text).strip().split(',')[0]
    price_get = re.search(r'\d+\.?\d*', price_get)
    # price_get = re.search(r'\d+\.?\d*', str(text))
    if price_get:
        try:
            return float(price_get.group())
        except:
            return np.nan
    return np.nan
# Получение курса валюты с сайта ЦБ
    # url = "https://www.cbr.ru/scripts/XML_daily.asp"
    # df = pd.read_xml(url, encoding='windows-1251') # нужна библиотека openpyxml 
    # usd_course = df[df['CharCode'] == 'USD']['Value'].iloc[0]
    # if pd.isna(usd_course):
    #     usd_course = 90.0
    # else:
    #     usd_course = float(str(usd_course).replace(',', '.'))

    # eur_course = df[df['CharCode'] == 'EUR']['Value'].iloc[0]
    # if pd.isna(eur_course):
    #     eur_course = 100.0
    # else:
    #     eur_course = float(str(eur_course).replace(',', '.'))

    # usd_course = 90.0
    # eur_course = 100.0


    # patterns = [
    # (r'(\d[\d\s\.]*)\s*(?:\$|долл|usd)', usd_course),      # доллары
    # (r'(\d[\d\s\.]*)\s*(?:€|евро|eur)', eur_course),     # евро
    # (r'(\d[\d\s\.]*)\s*(?:руб|р\.)', 1.0)           # рубли
    # ]
    
    # for price_get, course_val in patterns:
    #     match = re.search(price_get, text)
    #     if match:
    #         num_str = match.group(1).replace(' ', '').replace(',', '.')
    #         try:
    #             price_foreign = float(num_str)
    #             price_rub = price_foreign * course_val
    #             return price_rub
    #         except:
    #             continue

data_df['price_tmp'] = data_df['Цена'].apply(get_price_rent)

In [26]:
#  выделение площади
def get_area_rent(text):
    if pd.isna(text) or str(text).strip() == '':
        return np.nan
    area_get = str(text).strip().split('/')[0]
    if area_get:
        try:
            return float(area_get)
        except:
            return np.nan
    return np.nan

data_df['area_tmp'] = data_df['Площадь, м2'].apply(get_area_rent)

In [27]:
#  выделение количества комнат
def get_cnt_room_rent(text):
    if pd.isna(text) or str(text).strip() == '':
        return np.nan
    cnt_room_get = str(text).strip().split(',')[0]
    if cnt_room_get:
        try:
            return float(cnt_room_get)
        except:
            return np.nan
    return np.nan

data_df['count_room'] = data_df['Количество комнат'].apply(get_cnt_room_rent)

In [28]:
data_df.describe().round(2)

,ID объявления,"Высота потолков, м",price_tmp,area_tmp,count_room
count,1.973700e+04,10535.00,19737.00,19737.00,19202.00
mean,2.671149e+08,2.99,86563.85,63.56,2.03
std,1.980106e+07,7.85,128148.32,47.94,1.01
min,1.072986e+08,1.20,5000.00,7.00,1.00
25%,2.712212e+08,2.64,39990.00,38.90,1.00
50%,2.739284e+08,2.64,50000.00,49.80,2.00
75%,2.746973e+08,2.80,75000.00,70.00,3.00
max,2.750064e+08,320.00,3000000.00,811.00,6.00


Отмечаем выбросы в новых числовых переменных `price_tmp` и `area_tmp`, которые вероятно связаны с дорогими объектами. Нужно определить, как с ними работать.

Средняя цена аренды квартиры - 86.5 тыс. руб.

Средняя площадь - 63,5 м2

Среднее количество комнат - 2

## Распределение целевой переменной

Целевая переменная — месячная арендная плата в рублях.  


In [29]:
fig = px.histogram(data_df, 
                    x='price_tmp', 
                    nbins=200,
                    title='Распределение целевой переменной',
                    marginal='box',
                    labels={'price_tmp': 'Цена аренды (целевая переменная)'
                            } 
)

# Увеличиваем высоту — чтобы вместить все строки
fig.update_layout(height=800)

fig.show()


Распределение имеет скошенность вправо: основная масса объявлений до 200 тыс. руб.

Пик в пределах 40 - 60 тыс. руб.

Медиана 50 тыс. руб.

Есть длинный хвост дорогих объектов до 3 млн руб. Удаление дорогих объектов может повлиять в худшую сторону на качество предсказаний модели в дальнейшем. Рукомендуется применить нормирование (логарифирование например)


## Зависимость целевой переменной.

Кореляция целевой переменная с числовыми параметрами

In [30]:
corr_df = data_df.corr(numeric_only=True)
fig = px.imshow(
    corr_df,
    text_auto='.2f',
    color_continuous_scale='RdBu_r',
    title='Матрица корреляций'
)
fig.update_layout(height=800)
fig.show()

Целевая переменная `price_tmp` зависит от `площади квартиры` (area_tmp) - 0.75, а также от в меньшей степени от `количества комнат` (area_tmp) - 0.58.

Влияние высоты потолков минимально.

## Распределение количества комнат

In [31]:
fig = px.histogram(
    data_df,
    x='count_room',
    title='Распределение количества комнат',
    labels={'count_room': 'Количество комнат'},
    nbins=10
)

fig.update_layout(xaxis_title="Количество комнат", yaxis_title="Частота", height=800)
fig.show()

In [32]:
data_df['count_room'] \
    .value_counts(normalize=True) \
    .reset_index() \
    .rename(columns={
                'count_room': 'Количество комнат',
                'count': 'Доля'
                    }
            ) \
    .round(2)

,Количество комнат,proportion
0,2.0,0.39
1,1.0,0.35
2,3.0,0.19
3,4.0,0.05
4,5.0,0.02
5,6.0,0.01


data_df['count_room'].value_counts

Наибольшее количество объявлений (93%) это 1, 2х и 3х комнатные квартиры.

5 и 6 комнатные квартиры составляют примерно 3%.

## Зависимость целевой переменной от площади

In [33]:
fig = px.scatter(
    data_df,
    x='area_tmp',
    y='price_tmp',
    title='Зависимость цены от площади',
    labels={'area_tmp': 'Общая площадь (м²)', 'price_tmp': 'Цена (руб.)'},
    opacity=0.6
)

fig.update_traces(marker=dict(size=10))
fig.show()

На графике видна прямая линейная зависимость цены аренды от площади.

При этом есть выбросы, на которых при большой площади установлена низкая цена.

Необходимо провести анализ таких отклонений.

In [34]:
data_df[(data_df['area_tmp'] > 400) & (data_df['price_tmp'] < 200000)]

,ID объявления,Количество комнат,Тип,Метро,Адрес,"Площадь, м2",Дом,Парковка,Цена,Телефоны,Описание,Ремонт,"Площадь комнат, м2",Балкон,Окна,Санузел,Можно с детьми/животными,Дополнительно,Название ЖК,Серия дома,"Высота потолков, м",Лифт,Мусоропровод,Ссылка на объявление,price_tmp,area_tmp,count_room
2522,274633188,3,Квартира,м. Мякинино (15 мин пешком),"Москва, Неманский проезд, 1к1",775.0/10.0,5/17,NaN,"60000.0 руб./ За месяц, Залог - 60000 руб., Ко...",+79605090443,На длительный срок предлагаю в аренду чистую т...,Косметический,NaN,Балкон (1),NaN,Совмещенный (1),Можно с детьми,"Мебель в комнатах, Мебель на кухне, Стиральная...",NaN,NaN,2.64,Пасс (2),Да,https://www.cian.ru/rent/flat/274633188,60000.0,775.0,3.0
7109,274475586,"2, Изолированная",Квартира,м. Свиблово (10 мин пешком),"Москва, улица Амундсена, 1К2",560.0/49.0/9.0,"3/4, Кирпичный",NaN,"45000.0 руб./ За месяц, Залог - 45000 руб., Ко...","+79165686016, +79623602735",Двухкомнатная квартира от собственника \n\nСда...,Косметический,12-18,Балкон (1),Во двор,Раздельный (1),NaN,"Мебель в комнатах, Мебель на кухне, Ванна, Сти...",NaN,NaN,3.20,NaN,Нет,https://www.cian.ru/rent/flat/274475586,45000.0,560.0,2.0
13567,256630677,"5, Оба варианта",Квартира,м. Проспект Вернадского (16 мин пешком),"Москва, Ленинский проспект, 128К1",500.0/160.0/40.0,"6/30, Кирпичный",подземная,"9800.0 €/ За месяц, Залог - 9800 €, Коммунальн...",+79684299112,Без комиссии для арендатора. Предлагается роск...,NaN,80+15-25-20-20,Лоджия (3),На улицу и двор,Совмещенный (3),"Можно с детьми, Можно с животными","Мебель в комнатах, Ванна, Душевая кабина, Стир...","Квартал на Ленинском, 2001",индивидуальный проект,2.80,"Пасс (2), Груз (2)",NaN,https://www.cian.ru/rent/flat/256630677,9800.0,500.0,5.0
13870,274946067,2,Квартира,м. Беговая (2 мин пешком),"Москва, Хорошевское шоссе, 12к1",550.0/47.0/10.0,"21/40, Монолитный",NaN,"76000.0 руб./ За месяц, Залог - 76000 руб., Ко...",+79853809063,Очень уютная и тёплая квартира исключительно ...,Евроремонт,15,NaN,На улицу,Совмещенный (1),Можно с детьми,"Мебель в комнатах, Мебель на кухне, Ванна, Сти...",NaN,NaN,NaN,"Пасс (4), Груз (1)",Нет,https://www.cian.ru/rent/flat/274946067,76000.0,550.0,2.0
23141,229232386,6,Квартира,м. Сокол (9 мин на машине),"Москва, Иваньковское шоссе, 5",800.0/490.0/20.0,"23/24, Монолитно-кирпичный",подземная,"22000.0 $/ За месяц, Залог - 22000 $, Коммунал...","+79152972995, +79651454322",Лот 38410. ЖК АЛИСА. Предлагается в аренду мно...,Дизайнерский,100-30-35-30-35-70-20-50-60-60,Лоджия (3),Во двор,Совмещенный (4),"Можно с детьми, Можно с животными","Мебель в комнатах, Мебель на кухне, Стиральная...",NaN,NaN,NaN,"Пасс (2), Груз (2)",NaN,https://www.cian.ru/rent/flat/229232386,22000.0,800.0,6.0
23142,224335697,6,Квартира,м. Сокол (8 мин на машине),"Москва, Иваньковское шоссе, 5",800.0/640.0,23/23,NaN,"23000.0 $/ За месяц, Залог - 23000 $, Коммунал...",+79651074479,Лот: 32563. Екатерина. БОНУС! Вашему внимани...,Евроремонт,NaN,Балкон (1),Во двор,Совмещенный (4),"Можно с детьми, Можно с животными","Мебель в комнатах, Мебель на кухне, Ванна, Душ...",NaN,NaN,NaN,NaN,NaN,https://www.cian.ru/rent/flat/224335697,23000.0,800.0,6.0
23143,258952978,"6, Оба варианта",Квартира,м. Сокол (7 мин на машине),"Москва, Иваньковское шоссе, 5",800.0/410.0/29.0,23/23,подземная,"23000.0 $/ За месяц, Залог - 23000 $, Срок аре...","+79663231130, +79099269384",ID 28479: Пентхаус 800 кв.м представительског...,Евроремонт,NaN,NaN,На улицу и двор,Совмещенный (3),"Можно с детьми, Можно с животными","Мебель на кухне, Ванна, Посудомоечная машина, ...",NaN,NaN,3.20,Пасс (1),NaN,https://www.cian.ru/rent/flat/258952978,23000.0,800.0,6.0


При анализе датафрейма по условиям площадь квартиры > 400 и цена квартиры < 200000 установлено наличие цен в долларах США и Евро. А также вероятно ошибочное указание площадей.

В связи с незначительным количеством таких объявлений возможно их удалить или скорректировать цены с учетом курса и площадь.

## Зависимость цены аренды от количества комнат

In [35]:
fig = px.box(
    data_df,
    x='count_room',
    y='price_tmp',
    title='Цена аренды по количеству комнат',
    labels={'count_room': 'Количество комнат', 'price_tmp': 'Цена (руб.)'}
)
fig.update_layout(height=1200)
fig.show()

Отмечаем рост цены аренды с увеличенем количества комнат.

1, 2х и 3х комнатные имеют близкие диапазоны цен. С увеличением количества комнат рост цены больше, чем на меньшем количестве комнат.

Наличие выбросов цен, которые могут быть связаны с другими параметрами (ремонт, наполнение квартиры, район, наличие метро и т.д.).

Необходимо поработать над ними. Рекомендация провести нормализацию (например логарифмирование) или удалить (риск потерять качество метрик и предсказаний)

## Процент объявлений с парковкой / лифтом

In [36]:
# есть парковка или лифт
data_df['Наличие парковки'] = data_df['Парковка'].notna().astype(int)
data_df['Наличие лифта'] = data_df['Лифт'].notna().astype(int)
data_df['Наличие м-провода'] = data_df['Мусоропровод'].map({'Да': 1, 'Нет': 0}).fillna(0).astype(int)

counts = data_df[['Наличие парковки', 'Наличие лифта', 'Наличие м-провода']].sum() / len(data_df) * 100

fig = px.bar(
    x=counts.index,
    y=counts.values,
    title='Процент объявлений с ключевыми удобствами',
    labels={'x': 'Удобство', 'y': 'Процент (%)'}
)

fig.show()

Парковку имеют более 43% объявлений. Вероятно может повышать ценность объекта.

Лифт - более 78%. Возможно имеет ценность наличие грузового лифта и нескольких лифтов.

Мусоропровод - почти 52%.

Данные признаки стоит учесть в дальнейшей работе.

## Зависимость цены аренды от количества комнат и типа ремонта

In [37]:
mean_price_grp = data_df.groupby(['Ремонт', 'count_room']).agg(
    mean_price_rent=('price_tmp', 'mean')
    ).reset_index() \
    .rename(columns={
        'Ремонт': 'Вид ремонта',
        'count_room': 'Кол-во комнат',
        'mean_price_rent': 'Ср. цена аренды'
    })

fig = px.bar(
    mean_price_grp,
    x='Кол-во комнат',
    y='Ср. цена аренды',
    color='Вид ремонта',
    barmode='group',
    title='Средняя цена аренды по виду ремонта и количеству комнат',
    labels={'Ср. цена аренды': 'Средняя цена аренды (₽)', 'Кол-во комнат': 'Количество комнат'},
    text='Ср. цена аренды'
)

fig.update_traces(texttemplate='%{text:.0f}', textposition='outside')
fig.show()

Видна зависимость цены аренды от типа ремонта и количества комнат. 

Для 1-3 комнатных квартир тенденция схожа - самые дорогие по средней цене аренды имеют дизайнерский ремонт. 

Косметический и без ремонта примерно схожи по цене и имеют низкую среднюю цену.

5-6 комнатные квартиры также имеют высокую среднюю цену, но при этом показатель без ремонта вырастает, особенно в 6 комнатных.

Ремонт стоит учесть в дальнейшем.

## Метро и количество объявлений (локализаця ТОП-10)

In [38]:
metro_stantion = data_df['Метро'].str.split('(', n=1).str[0]
data_df['Станция'] = metro_stantion.str.replace('м.', '', regex=False).str.strip()
metro_grp = data_df['Станция'].value_counts() \
                                .reset_index() \
                                .rename(columns={
                                        'count': 'Кол-во объявлений'
                                })
metro_grp['Доля'] = (metro_grp['Кол-во объявлений'] / metro_grp['Кол-во объявлений'].sum() * 100).round(2)
metro_grp = metro_grp.head(10)
metro_grp

,Станция,Кол-во объявлений,Доля
0,Селигерская,393,2.03
1,Щелковская,313,1.61
2,Бабушкинская,284,1.46
3,Новогиреево,282,1.45
4,Коломенская,273,1.41
5,Водный стадион,220,1.13
6,Проспект Вернадского,214,1.10
7,Медведково,213,1.10
8,Крылатское,208,1.07
9,Сокол,208,1.07


In [39]:
fig = px.bar(
    metro_grp,
    x='Станция',
    y='Кол-во объявлений',
    title='Топ-10 станций метро по объявлениям',
    labels={'Станция': 'Станция метро'}
)
fig.update_layout(
    xaxis_tickangle=-45,
    height=800
)
fig.show()


На 1м месте станция метро Селигерская, на которую приходится 2% объявлени1 с указанием станции метро. Но возможно это не только пешая доступность, но и транспортная (квартира в московской области).

Вероятно нужно учесть время и тип доступности при анализе (10 минут пешком и 10 минут на транспорте могут влиять на цену).

Стоит учесть адрес квартиры для определения цены.

## Вывод:

### Целевая переменная — цена аренды:

- Средняя цена — 86.5 тыс. руб., медиана — 50 тыс. руб.
- Распределение очень скошено — большинство объявлений дешёвые, есть длинный хвост дорогих объектов (до 3 млн).
- Основная масса объявлений с ценой до 200 тыс. руб
- Цена сильно зависит от площади (корреляция 0.75) → чем больше площадь, тем дороже.
- Умеренная связь с количеством комнат (корреляция 0.58).
- Тип ремонта влияет на цену:  

    Дизайнерский — самый дорогой.  
    Косметический / без ремонта — самые дешёвые для 1/2/3 комнатных квартир.  
    В 5–6 комнатных квартирах "без ремонта" иногда дороже — странно, нужно проверить.

### Дополнительные факторы:

- Парковка — есть у 43% → может повышать цену.
- Лифт — у 78% → наличие грузового или нескольких лифтов — плюс, но в целом наличие лифта спорно влияет на цену.
- Мусоропровод — почти у половины → тоже стоит учесть.
- Метро — лидер по частоте: Селигерская (2%), но важно учитывать как далеко (пешком или на транспорте).
- Адрес — ключевой фактор → поможет определить район и цену.

### Есть грязные и неинформацтивные данные в датафрейме:
- присутствуют объявления не из Москвы
- колонка Тип имеет одно значение и неинфорацтивна
- Серия дома и Название ЖК — слишком много пропусков (>70%) и неинформативна, рекомендация удалить
- выбросы (Высота потолков — 1.2 м и 320 м → явные ошибки) 

### Проблемы и рекомендации:

- 
- Не удалять выбросы просто так — они могут быть реальными (дорогие квартиры в центре, с ремонтом и видом). Лучше логарифмировать.
- Проверить данные по типу ремонта, парковке, метро — они важны для модели.
- Убедиться, что все цены в рублях — исправить или удалить те, что в валюте.

## Итог:

    Данные “грязные”, но полезные. Нужно почистить, выделить числовые признаки, логарифмировать цену, удалить бесполезные колонки. Цена зависит от площади и комнат, но также важны ремонт, парковка, метро и адрес. Возможно и другие признаки.